In [1]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

# Load Data

In [2]:
data_path = pathlib.Path('../data')

In [3]:
paper_data = pd.concat([
    pd.read_csv(data_path / 'gidden_et_al_2022_ar6_reanalysis_data.csv'),
    pd.read_csv(data_path / 'additional_analysis_data.csv'),
])

In [4]:
df = pyam.IamDataFrame(paper_data)
df.load_meta(data_path / 'gidden_et_al_2022_ar6_reanalysis_meta.csv')

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading meta indicators


In [5]:
variables = ['Emissions|CO2', 'Emissions|Kyoto Gases']
ar6_data = pd.read_csv(data_path / 'AR6_Scenarios_Database_World_v1.0.csv')
ar6_data = ar6_data[ar6_data.Variable.isin(variables)]
df.append(pyam.IamDataFrame(ar6_data).filter(index=df.index), inplace=True)

In [6]:
variable_templates = [
    'Emissions|{gas}', # original from AR6
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}|Direct Only', # reanalyzed by OSCAR
    'AR6 Reanalysis|OSCARv3.2|Emissions|{gas}', # reanalyzed by OSCAR including NGHGI adjustment 
]

# Compute Carbon Budgets

In [7]:
def make_carbon_budget(df, variable):
    meta = (
        df
        .meta
        .query("Category in ['C1', 'C3']")
        [f'Interpolated Net-Zero Year for {variable}']
        .dropna()
        .astype(int)
        .reset_index()
    ) 
    data = (
        df
        .filter(region='World', variable=f'Cumulative {variable} from 2020')
        .convert_unit('Mt CO2/yr', to='Gt CO2', factor=1e-3)
    )
    budgets = [(
        row['model'], 
        row['scenario'], 
        data.filter(
            model=row['model'], scenario=row['scenario'], 
            year=row[f'Interpolated Net-Zero Year for {variable}']
        ).timeseries().values[0][0]
    )
        for i, row in meta.iterrows()
    ]
    return pd.DataFrame(budgets, columns=['model', 'scenario', f'Carbon Budget for {variable}']).set_index(['model', 'scenario'])
        

In [8]:
new_meta = pd.concat([
    make_carbon_budget(df, template.format(gas='CO2')) for template in variable_templates
], axis=1)
df.set_meta(new_meta)
new_meta.head()

Carbon Budget for Emissions|CO2  \
model       scenario                                                 
AIM/CGE 2.0 SSP1-26                                     988.478191   
            SSP4-26                                    1099.938530   
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                       630.020630   
            CD-LINKS_NPi2020_400                        470.828982   
AIM/CGE 2.2 EN_INDCi2030_1000f                         1135.329505   

                                   Carbon Budget for AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  \
model       scenario                                                                                      
AIM/CGE 2.0 SSP1-26                                                       862.102117                      
            SSP4-26                                                      1036.223839                      
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         493.573297                      
            CD-LINKS_NPi2020_400                                          364.560104                      
AIM/CGE 2.2 EN_INDCi2030_1000f                                            989.933137                      

                                   Carbon Budget for AR6 Reanalysis|OSCARv3.2|Emissions|CO2  
model       scenario                                                                         
AIM/CGE 2.0 SSP1-26                                                       711.308853         
            SSP4-26                                                       876.356706         
AIM/CGE 2.1 CD-LINKS_NPi2020_1000                                         374.487241         
            CD-LINKS_NPi2020_400                                          298.821486         
AIM/CGE 2.2 EN_INDCi2030_1000f                                            820.869212

# Emission Reductions

In [9]:
def compute_reduction(df, variable, fromyear=2020, toyear=2030):
    data = (
        df
        .filter(variable=variable, year=[fromyear, toyear], region='World')
        .timeseries()
        .droplevel(['region', 'unit', 'variable'])
    )
    s = (data[fromyear] - data[toyear]) / data[fromyear] * 100
    s.name = f'2030 Emission Reductions for {variable}'
    return s

In [10]:
gases = ['CO2', 'Kyoto Gases']
new_meta = pd.concat([
    compute_reduction(df, template.format(gas=gas))
    for gas, template in itertools.product(gases, variable_templates)
], axis=1)
df.set_meta(new_meta)
new_meta.head()

2030 Emission Reductions for Emissions|CO2  \
model       scenario                                                    
AIM/CGE 2.0 SSP1-26                                         19.877794   
            SSP1-34                                          5.324050   
            SSP1-45                                         -7.844705   
            SSP1-Baseline                                   -7.004407   
            SSP4-26                                         21.978920   

                           2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|CO2|Direct Only  \
model       scenario                                                                                         
AIM/CGE 2.0 SSP1-26                                                20.690203                                 
            SSP1-34                                                 3.454514                                 
            SSP1-45                                                -6.983672                                 
            SSP1-Baseline                                          -8.264738                                 
            SSP4-26                                                19.278438                                 

                           2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|CO2  \
model       scenario                                                                             
AIM/CGE 2.0 SSP1-26                                                24.188745                     
            SSP1-34                                                 5.676354                     
            SSP1-45                                                -5.455350                     
            SSP1-Baseline                                          -6.693626                     
            SSP4-26                                                22.529429                     

                           2030 Emission Reductions for Emissions|Kyoto Gases  \
model       scenario                                                            
AIM/CGE 2.0 SSP1-26                                                24.882195    
            SSP1-34                                                 9.073330    
            SSP1-45                                                -6.835497    
            SSP1-Baseline                                          -7.861319    
            SSP4-26                                                27.685739    

                           2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases|Direct Only  \
model       scenario                                                                                                 
AIM/CGE 2.0 SSP1-26                                                25.684139                                         
            SSP1-34                                                 7.263869                                         
            SSP1-45                                                -6.940654                                         
            SSP1-Baseline                                          -9.472901                                         
            SSP4-26                                                26.476508                                         

                           2030 Emission Reductions for AR6 Reanalysis|OSCARv3.2|Emissions|Kyoto Gases  
model       scenario                                                                                    
AIM/CGE 2.0 SSP1-26                                                28.421538                            
            SSP1-34                                                 9.035403                            
            SSP1-45                                                -5.887162                            
            SSP1-Baseline                                          -8.470633                            
            SSP4-26             

In [11]:
def summarize(data, variable_start, gas, name):
    cols = [variable_start + template.format(gas=gas) for template in variable_templates]
    headers = ['Original Pathways', 'Pathways with OSCAR Direct Effects', 'Pathways with OSCAR Direct & Indirect Effects']
    for col in cols:
        assert col in data
    return (
        data
        .query("Category in ['C1', 'C3']")
        .replace({'Category': {'C1': '1.5C', 'C3': '2C'}})
        .rename(columns={c: h for c, h in zip(cols, headers)})
        [headers + ['Category']]
        .groupby('Category')
        .describe(percentiles=(0.1, 0.5, 0.9))
        .stack(level=-2)
        [['50%', '10%', '90%']]
        .astype(int)
        .apply(lambda col: f"{col['50%']} ({col['10%']}-{col['90%']})", axis=1)
        .to_frame(name=name)
        .loc[['1.5C', '2C'], headers, :]
    )

In [12]:
to_summarize = {
    ('Carbon Budget for ', 'CO2'): 'Carbon Budget from 2020',
    ('2030 Emission Reductions for ', 'CO2'): 'CO2 Emissions Reductions (2020-2030)',
    ('Interpolated Net-Zero Year for ', 'CO2'): 'Net-zero CO2 Year',
    ('Interpolated Net-Zero Year for ', 'Kyoto Gases'): 'Net-zero GHG Year',
}

In [13]:
table = pd.concat([
    summarize(df.meta, variable_start, gas, name)
    for (variable_start, gas), name in to_summarize.items()
], axis=1).T
table.to_excel('mitigation_outcomes.xlsx')
table

Category                                          1.5C  \
                                     Original Pathways   
Carbon Budget from 2020                  545 (398-630)   
CO2 Emissions Reductions (2020-2030)        49 (37-63)   
Net-zero CO2 Year                     2055 (2045-2067)   
Net-zero GHG Year                     2071 (2054-2087)   

Category                                                                 \
                                     Pathways with OSCAR Direct Effects   
Carbon Budget from 2020                                   473 (375-562)   
CO2 Emissions Reductions (2020-2030)                         52 (37-64)   
Net-zero CO2 Year                                      2050 (2042-2058)   
Net-zero GHG Year                                      2066 (2054-2085)   

Category                                                                            \
                                     Pathways with OSCAR Direct & Indirect Effects   
Carbon Budget from 2020                                              392 (319-480)   
CO2 Emissions Reductions (2020-2030)                                    56 (41-70)   
Net-zero CO2 Year                                                 2047 (2039-2052)   
Net-zero GHG Year                                                 2067 (2050-2083)   

Category                                            2C  \
                                     Original Pathways   
Carbon Budget from 2020                 908 (665-1095)   
CO2 Emissions Reductions (2020-2030)         21 (1-41)   
Net-zero CO2 Year                     2070 (2060-2085)   
Net-zero GHG Year                     2083 (2076-2091)   

Category                                                                 \
                                     Pathways with OSCAR Direct Effects   
Carbon Budget from 2020                                  838 (614-1032)   
CO2 Emissions Reductions (2020-2030)                          21 (1-44)   
Net-zero CO2 Year                                      2068 (2057-2081)   
Net-zero GHG Year                                      2082 (2072-2093)   

Category                                                                            
                                     Pathways with OSCAR Direct & Indirect Effects  
Carbon Budget from 2020                                              703 (494-865)  
CO2 Emissions Reductions (2020-2030)                                     25 (4-48)  
Net-zero CO2 Year                                                 2064 (2053-2078)  
Net-zero GHG Year                                                 2082 (2068-2094)

# Net-zero timings compared to a constant offset of 3.7 Gt CO2

Note that all pathways are included in this calculation whereas we only include scenarios available for assessment by OSCAR above.

- C1-C3: 2067 (2051-2100) to 2058 (2045-2069)
- C1: 2055 (2043-2070) to 2047 (2039-2055)
- C3: 2074 (2064-2100) to 2063 (2055-2072)